# Assigment 6

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

1. Generate a folder with the name of your group in this folder **GitHub\Diplomado_PUCP\_output\sbs\municipal**. The code should run even if the folder exists.
2. Scrape all the available information in this [link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C). Save an excel file by year and month request. The excel file should be name using the month and year of the sheet. 

In [3]:
!pip install selenium
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [31]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
import numpy as np
from selenium.webdriver.common.by import By
import pandas as pd

In [5]:
## Link 
link= 'https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C'

### 1. Maximize browser

In [35]:
driver = webdriver.Chrome( ChromeDriverManager().install() )

url = 'https://www.kaspersky.com/resource-center/definitions/cookies'
driver.get( url )

driver.maximize_window()

C:\Users\jorge\AppData\Local\Temp\ipykernel_31416\1931229864.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome( ChromeDriverManager().install() )


### 2. Set the browser 100%

In [36]:
driver.execute_script("document.body.style.zoom='100%'")

### 3. Web Scrapping

In [37]:
driver.get( link )

In [9]:
## Construir index
index = driver.find_element( By.XPATH, "/html/body/form/div[4]/div/div/div[2]/div/div/div/div[1]/div/div/table/tbody" )
list_index= index.find_elements( By.TAG_NAME, "td")
newlist = [i.text for i in list_index]
new =newlist[-47:-1]
pd_index = pd.DataFrame(new,columns =['Tasa Anual (%)'])

In [15]:
# Tabla principal
tabla1 = driver.find_element( By.XPATH, "/html/body/form/div[4]/div/div/div[2]/div/div/div/div[1]/div/div/table/tbody" )
table_html = tabla1.get_attribute( 'outerHTML' )
table = pd.read_html( table_html)
New_df = pd.concat([table[0],pd_index], axis=1)
Final = New_df.set_index('Tasa Anual (%)')

In [18]:
Final

,CMAC Arequipa,CMAC Cusco,CMAC del Santa,CMAC Huancayo,CMAC Ica,CMAC Maynas,CMAC Paita,CMAC Piura,CMAC Sullana,CMAC Tacna,CMAC Trujillo,CMCP Lima,Promedio
Tasa Anual (%),,,,,,,,,,,,,
Corporativos,-,-,-,-,-,12.01,-,-,9.23,-,12.00,-,9.95
Descuentos,-,-,-,-,-,-,-,-,-,-,-,-,-
Préstamos hasta 30 días,-,-,-,-,-,-,-,-,-,-,-,-,-
Préstamos de 31 a 90 días,-,-,-,-,-,-,-,-,-,-,-,-,-
Préstamos de 91 a 180 días,-,-,-,-,-,-,-,-,-,-,-,-,-
Préstamos de 181 a 360 días,-,-,-,-,-,-,-,-,9.00,-,-,-,9.00
Préstamos a más de 360 días,-,-,-,-,-,12.01,-,-,10.00,-,12.00,-,11.21
Grandes Empresas,8.79,-,-,-,-,-,-,-,-,-,-,-,8.79
Descuentos,-,-,-,-,-,-,-,-,-,-,-,-,-


In [111]:
# Dropdown button list
dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio")
dropdown_button_year.click()

dropdown_button_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes")
dropdown_button_month.click()

# List of years options
list_year = driver.find_element( By.ID,  "ctl00_cphContent_rAnio_DropDown" )
years_list = list_year.find_elements( By.TAG_NAME, 'li')
list_year_options = len(years_list)

# List of months options
list_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes_DropDown" )
months_list = list_month.find_elements( By.TAG_NAME, 'li')
list_month_options = len(months_list)

driver.find_element( By.ID, "ctl00_cphContent_btnConsultaMensual").click()
driver.get('https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C')

In [114]:
year_numbers_opt  = np.array( [1, 0 ] + np.arange(2, list_year_options).tolist() )
month_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_month_options).tolist() )

In [121]:
years_info_list  = []
months_info_list = []
tables_info_list = []

for year_number in year_numbers_opt[0:23]:
    
    # Dropdown button list
    dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio") 
    dropdown_button_year.click()

    # List of years options
    time.sleep(0.5)
    driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{year_number + 1 }]" ).click()
    year_text = driver.find_element( By.ID, "ctl00_cphContent_rAnio").text

    
        
    for month_number in month_numbers_opt[0:13]:
        
        # Dropdown button list
        dropdown_button_month = driver.find_element( By.ID, "ctl00_cphContent_rMes") 
        dropdown_button_month.click()

        # # List of months options
        time.sleep(0.5)
        driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{month_number + 1 }]" ).click()
        month_text = driver.find_element( By.ID, "ctl00_cphContent_rMes").text
        print( year_text )
        print( month_text )
        
        # Click consultar
        driver.find_element( By.ID,"ctl00_cphContent_btnConsultaMensual").click()
        time.sleep(5)
        
        year_float = int(year_text)
        
        try:
            # Switch elements
            driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # Cleaning tablr
            table_clean_1 = table_pd[1].iloc[5:, :].copy().reset_index( drop = True )

            # Changing switch
            driver.switch_to.default_content()

            print("Table Tipo 1 - Cambiar de Frame y una tabla simple")

        except:
            print("No table Tipo 1")
            table_clean_1 = "Null"
            
            
        try:
            # Switch elements
            driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # generate table objects
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_2 = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_2.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 2 - Cambiar de Frame y una tabla Compleja")
            
             # Changing switch
            driver.switch_to.default_content()
            
        except:
            print("No table Tipo 2")
            table_clean_2 = "Null"
            

        try:
            # Get tables
            section_id_table = driver.find_element( By.ID, "ctl00_cphContent_rpgActualMn_OT" )
            table_html = section_id_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # generate table objects
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_3 = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_3.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 3 - No Cambiar de Frame y una tabla Compleja")
            
        except:
            print("No table Tipo 3")
            table_clean_3 = "Null"
            
            
        # Check what to save
        if ( ( isinstance( table_clean_3, str ) == False ) and ( table_clean_3.shape[1] > 5 ) ):
            
            # Data with no frame
            table_clean_3.to_excel( fr"../../../Diplomado_PUCP\_output\sbs\municipal\group_6\table_clean_{year_text}_{month_text}.xlsx", index = False )

            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 3" )
            print("guardo3")
            
        elif ( ( isinstance( table_clean_2, str ) == False ) and ( table_clean_2.shape[1] > 5 ) ):
            
            # Data with frame and 2 tables
            table_clean_2.to_excel( fr"../../../Diplomado_PUCP\_output\sbs\municipal\group_6\table_clean_{year_text}_{month_text}.xlsx", index = False )
            # To break
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 2" )
            print("guardo2")
            
        elif ( ( isinstance( table_clean_1, str ) == False ) and ( table_clean_1.shape[1] > 5 ) ):
            # Data with frame and 1 table
            table_clean_1.to_excel( fr"../../../Diplomado_PUCP\_output\sbs\municipal\group_6\table_clean_{year_text}_{month_text}.xlsx", index = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 1" )
            print("guardo1")
            
        # No table information
        # Go to a previous day
        else:
            # Day ago
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "No Table" )
            
        driver.switch_to.default_content()
                

2022
Febrero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Enero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Marzo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Abril
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Mayo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Junio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Julio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Agosto
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Setiembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Octubre
No table Tipo 1
No

2017
Octubre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2017
Noviembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2017
Diciembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Febrero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Enero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Marzo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Abril
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Mayo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Junio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Julio
No table Tipo 1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/form/div[1]/div/ul/li[10]"}
  (Session info: chrome=109.0.5414.121)
Stacktrace:
Backtrace:
	(No symbol) [0x00846643]
	(No symbol) [0x007DBE21]
	(No symbol) [0x006DDA9D]
	(No symbol) [0x00711342]
	(No symbol) [0x0071147B]
	(No symbol) [0x00748DC2]
	(No symbol) [0x0072FDC4]
	(No symbol) [0x00746B09]
	(No symbol) [0x0072FB76]
	(No symbol) [0x007049C1]
	(No symbol) [0x00705E5D]
	GetHandleVerifier [0x00ABA142+2497106]
	GetHandleVerifier [0x00AE85D3+2686691]
	GetHandleVerifier [0x00AEBB9C+2700460]
	GetHandleVerifier [0x008F3B10+635936]
	(No symbol) [0x007E4A1F]
	(No symbol) [0x007EA418]
	(No symbol) [0x007EA505]
	(No symbol) [0x007F508B]
	BaseThreadInitThunk [0x76A46BD9+25]
	RtlGetFullPathName_UEx [0x77568FD2+1218]
	RtlGetFullPathName_UEx [0x77568F9D+1165]
